In [51]:
import simpy
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from ipywidgets import interact
import random

TOTAL_RUNTIME=14400  # 4 hours

# Function to run the simulation with adjustable parameters
def run_simulation(min_idle_consumers, consumer_startup_delay, consumer_busy_time, max_consumers,
                   work_burst_size, consumer_start_percentage, check_interval, burst_count):
    # Data storage for plotting
    global idle_consumers_data, busy_consumers_data, pending_consumers_data, work_burst_data
    idle_consumers_data = []
    busy_consumers_data = []
    pending_consumers_data = []
    work_burst_data = []

    class Consumer:
        def __init__(self, env, queue):
            self.env = env
            self.queue = queue
            self.state = 'pending'
            self.action = env.process(self.pending())

        def pending(self):
            self.queue.pending.append(self)
            yield self.env.timeout(consumer_startup_delay * random.uniform(1, 1.25))
            self.queue.pending.remove(self)
            self.queue.idle.append(self)
            self.action = self.env.process(self.work())

        def work(self):
            while True:
                # Wait for work
                work_item = yield self.queue.store.get()
                self.queue.idle.remove(self)
                self.queue.processing.append(self)
                # Process work
                yield self.env.timeout(consumer_busy_time * random.uniform(0.75, 1.25))
                # Finish work and terminate
                self.queue.processing.remove(self)
                break

    class WorkQueue:
        def __init__(self, env):
            self.env = env
            self.idle = []
            self.processing = []
            self.pending = []
            self.store = simpy.Store(env)
            self.setup_initial_consumers()

        def setup_initial_consumers(self):
            for _ in range(min_idle_consumers):
                self.add_pending_consumer()

        def add_pending_consumer(self):
            Consumer(self.env, self)

        def add_work(self, count):
            for _ in range(count):
                self.store.put("work")
            work_burst_data.append(count)

    def generate_work(env, queue):
        burst_times = sorted(random.uniform(0, TOTAL_RUNTIME) for _ in range(burst_count))
        for burst_time in burst_times:
            yield env.timeout(burst_time - env.now)  # Wait until the burst time
            queue.add_work(work_burst_size)

    def monitor_consumers(env, queue):
        while True:
            yield env.timeout(check_interval)
            idle_consumers_data.append(len(queue.idle))
            busy_consumers_data.append(len(queue.processing))
            pending_consumers_data.append(len(queue.pending))
            total_consumers = len(queue.idle) + len(queue.processing)

            if len(queue.idle) < min_idle_consumers:
                consumers_to_start = max(int(total_consumers * consumer_start_percentage), 1)
                # start up to the max consumers, no more.
                consumers_to_start = min(max_consumers - total_consumers, consumers_to_start)
                for _ in range(consumers_to_start):
                    queue.add_pending_consumer()

    # Setup simulation environment
    env = simpy.Environment()
    queue = WorkQueue(env)

    # Start processes
    env.process(generate_work(env, queue))
    env.process(monitor_consumers(env, queue))

    env.run(until=TOTAL_RUNTIME)

    # Plotting
    time = np.arange(0, len(idle_consumers_data) * check_interval, check_interval) / 60

    plt.figure(figsize=(15, 5))
    plt.stackplot(time, idle_consumers_data, busy_consumers_data, labels=['Idle Consumers', 'Busy Consumers'])
    plt.legend(loc='upper right')
    plt.xlabel('Time (minutes)')
    plt.ylabel('Number of Consumers')
    plt.title('Idle and Busy Consumers Over Time')
    plt.show()

    plt.figure(figsize=(15, 5))
    plt.plot(time, pending_consumers_data, label='Pending Consumers')
    plt.legend(loc='upper right')
    plt.xlabel('Time (minutes)')
    plt.ylabel('Number of Pending Consumers')
    plt.title('Pending Consumers Over Time')
    plt.show()

# Interactive sliders for parameters
interact(run_simulation,
         min_idle_consumers=widgets.IntSlider(value=40, min=0, max=100, step=1, description='Min Idle Consumers'),
         max_consumers=widgets.IntSlider(value=500, min=0, max=1000, step=1, description='Max Consumers'),
         consumer_startup_delay=widgets.IntSlider(value=200, min=60, max=300, step=30, description='Startup Delay (s)'),
         consumer_busy_time=widgets.IntSlider(value=600, min=60, max=1200, step=100, description='Busy Time (s)'),
         work_burst_size=widgets.IntSlider(value=80, min=1, max=200, step=1, description='Burst Size'),
         consumer_start_percentage=widgets.FloatSlider(value=0.50, min=0.01, max=2, step=0.01, description='Start %'),
         check_interval=widgets.IntSlider(value=100, min=10, max=300, step=1, description='Check Interval (s)'),
         burst_count=widgets.IntSlider(value=10, min=0, max=100, step=1, description='bursts')
        )


interactive(children=(IntSlider(value=40, description='Min Idle Consumers'), IntSlider(value=200, description=…

<function __main__.run_simulation(min_idle_consumers, consumer_startup_delay, consumer_busy_time, max_consumers, work_burst_size, consumer_start_percentage, check_interval, burst_count)>